In [ ]:
%load_ext autoreload
%autoreload 2

# Tabulate and plot APF values

In [ ]:
import pandas as pd
import glob
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms

plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'arial'


In [ ]:
files = [Path(f) for f in sorted(glob.glob('../data/processed/peak_shapes/ht_area_tables/*.csv'))]

names = [f.stem for f in files]
apf_tables = [pd.read_csv(f, index_col=0) for f in files]
apf_tables = [df.set_index('Sample') for df in apf_tables]

print(names)


for df in apf_tables:
    df['ht_area_ratio_cf_GaN'] = df['ht_area_ratio'] / df.loc['GaN', 'ht_area_ratio']
    df['ht_area_ratio_cf_GaN_stdev'] = df['ht_area_ratio_mc_stdev'] / df.loc['GaN', 'ht_area_ratio']
    
    df['ht_area_ratio_cf_BN'] = df['ht_area_ratio'] / df.loc['BN', 'ht_area_ratio']
    df['ht_area_ratio_cf_BN_stdev'] = df['ht_area_ratio_mc_stdev'] / df.loc['BN', 'ht_area_ratio']
    
apf_tables[0]

In [ ]:
y = 'ht_area_ratio_cf_GaN'
y_sd = 'ht_area_ratio_cf_GaN_stdev'

combined_apf_table = pd.concat([df[y] for df in apf_tables], axis=1)
combined_stdev_table = pd.concat([df[y_sd] for df in apf_tables], axis=1)

combined_apf_table.columns = names
combined_stdev_table.columns = names


In [ ]:
combined_apf_table

In [ ]:
row_order = ['GaN', 'AlN', 'BN', 'Si3N4', 'GlassA870', 'Buddingtonite']
combined_apf_table = combined_apf_table.reindex(row_order, axis=0)
combined_stdev_table = combined_stdev_table.reindex(row_order, axis=0)

In [ ]:
col_order = ['ANU_LDE1', 'ANU_LDE1_stdev', 'ANU_LDE1H', 'ANU_LDE1H_stdev', 'StA_LDE1', 'StA_LDE1_stdev', 'StA_LDE1L', 'StA_LDE1L_stdev']
formatted_table = combined_apf_table.join(combined_stdev_table, rsuffix='_stdev')
formatted_table = formatted_table.reindex(col_order, axis=1)

In [ ]:
formatted_table.rename(
                    {col: col.replace('_', ' ') for col in formatted_table.columns}, axis=1, inplace=True)

formatted_table.rename(
                    {col: '±' for col in formatted_table.columns if 'stdev' in col}, axis=1, inplace=True)

In [ ]:
formatted_table['Average'] = formatted_table.loc[:,['ANU LDE1', 'ANU LDE1H', 'StA LDE1', 'StA LDE1L']].mean(axis=1)
formatted_table['StDev'] = formatted_table.loc[:,['ANU LDE1', 'ANU LDE1H', 'StA LDE1', 'StA LDE1L']].std(axis=1)
formatted_table = formatted_table.round(2)
formatted_table.fillna('-', inplace=True)
formatted_table.to_csv('../data/processed/peak_shapes/apf_relative_to_GaN.csv')
formatted_table

In [ ]:
fig, ax = plt.subplots(figsize=(8*0.39,8*0.39))

clrs = {'GaN':'tab:blue', 'BN':'tab:orange', 'Buddingtonite':'mediumaquamarine',
        'AlN': 'tab:purple', 'GlassA870': 'tab:red', 'Si3N4':'tab:cyan'}
symbols = {'SXES': '*', 'ANU_LDE1': '^', 'ANU_LDE1H': 'v', 'StA_LDE1': '<', 'StA_LDE1L': '>'}

for i, col in enumerate(combined_apf_table.columns):
    for j, sample in enumerate(combined_apf_table.index):
        
        if col == 'SXES':
            size = 20
        else:
            size = 10
        
        if j == 0:
            leg = col
        else:
            leg = '_nolegend'
        
        plt.errorbar(x=j + i*0.1,
                     y=combined_apf_table.loc[sample, col],
                     yerr=combined_stdev_table.loc[sample, col],
                     fmt=symbols[col], label=leg, color=clrs[sample],
                     markersize=size, markeredgecolor='k')

ax.set_xticks(np.arange(len(combined_apf_table)))
ax.set_xticklabels(labels=combined_apf_table.index, rotation=75)

plt.legend(fontsize=8)
plt.ylabel('Ht/Area relative to GaN')
plt.tight_layout()

# Get the actual data and fit to plot figures showing the peak shapes

Import data

In [ ]:
data = {'StA': 
            {'LDE1': {}, 'LDE1L': {}}
        ,'ANU': 
            {'LDE1': {}, 'LDE1H': {}}
       }

for lab in data.keys():
    for xtl in data[lab].keys():
        files = sorted(glob.glob(f'../data/processed/peak_shapes/data_with_fits_{lab}/{xtl}/*.csv'))
        samplenames = [Path(f).stem for f in files]
        spectral_data = [pd.read_csv(f, index_col=0) for f in files]
        data[lab][xtl] = dict(zip(samplenames, spectral_data))
        
        for n in samplenames:
            area = np.trapz(data[lab][xtl][n]['fit'], x=data[lab][xtl][n]['L'])
            data[lab][xtl][n]['cps_per_nA_corrected_scaled'] = data[lab][xtl][n]['cps_per_nA_corrected'] / area
            data[lab][xtl][n]['fit_scaled'] = data[lab][xtl][n]['fit'] / area


In [ ]:
lab = 'ANU'
xtl = 'LDE1H'

fig, ax = plt.subplots(figsize=(8*0.39, 8*0.39))

for n in row_order:
    data[lab][xtl][n].plot(x='L', y='fit_scaled', color=clrs[n], linestyle='-', label=n.replace('nite',"."), ax=ax, lw=1.5)
    data[lab][xtl][n].plot(x='L', y='cps_per_nA_corrected_scaled', color=clrs[n], linestyle='-', label='_nolegend', ax=ax, alpha=0.3, lw=0.5)
    
plt.xlim(130, 160)
plt.yticks([])
# plt.ylim(-0.01, 0.3)
plt.legend(fontsize=8, frameon=False)
plt.tight_layout()


In [ ]:
def plot_panel1(data, ax):

    lab = 'ANU'
    xtl = 'LDE1H'

    for n in row_order:
        data[lab][xtl][n].plot(x='L', y='fit_scaled', color=clrs[n], linestyle='-', ax=ax, lw=1.5, legend=False)
        data[lab][xtl][n].plot(x='L', y='cps_per_nA_corrected_scaled', color=clrs[n], linestyle='-', ax=ax, alpha=0.3, lw=0.5, legend=False)

    ax.set_xlim(137, 154)
    ax.set_ylabel('normalised cps/nA')
    ax.set_xlabel('L (mm)')
    # ax.set_yticks([])
    ax.set_ylim(-0.01, 0.25)
    # ax.legend(fontsize=8, frameon=False)
    
    return None

def plot_panel2(combined_apf_table, combined_stdev_table, ax):

    clrs = {'GaN':'tab:blue', 'BN':'tab:orange', 'Buddingtonite':'mediumaquamarine',
            'AlN': 'tab:purple', 'GlassA870': 'tab:red', 'Si3N4':'tab:cyan'}
    symbols = {'ANU_LDE1': '^', 'ANU_LDE1H': 'v', 'StA_LDE1': '<', 'StA_LDE1L': '>'}

    for i, col in enumerate(combined_apf_table.columns):
        for j, sample in enumerate(combined_apf_table.index):
            
            size = 10

            plt.errorbar(x=j + i*0.1,
                         y=combined_apf_table.loc[sample, col],
                         yerr=combined_stdev_table.loc[sample, col],
                         fmt=symbols[col], color=clrs[sample],
                         markersize=size, markeredgecolor='k')

    ax.set_xticks(np.arange(len(combined_apf_table)))
    ax.set_xticklabels(labels=combined_apf_table.index, rotation=75)
    
    for i, k in enumerate(symbols.keys()):
        ax.plot(5, 0.92 + 0.02*i, symbols[k], markersize = 10, markeredgecolor='grey', markerfacecolor='w')
        ax.text(4.7, 0.92 + 0.02*i, k.replace('_', ' '), va='center', ha='right', fontsize=8)
    
    # ax.legend(fontsize=8)
    ax.set_ylabel('Area-peak factor')

In [ ]:
fig = plt.figure(figsize=(8*0.39, 16*0.39))
mosaic = """
    A
    B
    """
ax_dict = fig.subplot_mosaic(mosaic)

plot_panel1(data, ax_dict['A'])
plot_panel2(combined_apf_table, combined_stdev_table, ax_dict['B'])

for label, ax in ax_dict.items():
    trans = mtransforms.ScaledTranslation(-5/72, -5/72, fig.dpi_scale_trans)
    ax.text(1.0, 1.0, label, transform=ax.transAxes + trans,
            fontsize=12, ha='right', va='top', fontfamily='Arial',
            bbox=dict(facecolor='w', edgecolor='grey', pad=3.0))

plt.tight_layout()
plt.savefig("../figures/peak_shape_and_apf_combined.pdf")

In [ ]:
combined_apf_table.round(3)

In [ ]:
combined_stdev_table.round(3)

In [ ]:
def format_table_entry(apf, apf_sd):
    apf_sd_last_digit = str(round(apf_sd, 3)).replace('.','').lstrip('0')
    string_out = '{:.3f}({})'.format(apf, apf_sd_last_digit)
    
    if string_out == 'nan(nan)':
        string_out = '-'
    return string_out

In [ ]:
formatted_apf_table = pd.DataFrame().reindex_like(combined_apf_table)

for col in formatted_apf_table.columns:
    for row in formatted_apf_table.index:
        formatted_apf_table.loc[row, col] = format_table_entry(combined_apf_table.loc[row, col],
                                                               combined_stdev_table.loc[row, col])

formatted_apf_table.columns = [col.replace('_', ' ') for col in formatted_apf_table.columns]
formatted_apf_table.to_csv("formatted_apf_table.csv")
formatted_apf_table